# Load Overture Data
## Query names and categories w/o bounding box from overture

In [3]:
import pandas as pd
%pip install --quiet duckdb geopandas shapely
import geopandas as gpd
import duckdb
from shapely import wkt
import csv

1389.36s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Note: you may need to restart the kernel to use updated packages.


In [ ]:
with open('data/overture_categories.csv', 'r') as f:
    categories = pd.read_csv(f)  # Use the already opened file object

print("Loading data from DuckDB...")
print(categories.to_string())

ParserError: Expected 2 fields in line 4, saw 3

In [ ]:
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

In [ ]:
%load_ext sql
conn = duckdb.connect()
%sql conn --alias duckdb

%sql INSTALL spatial;
%sql INSTALL httpfs;
%sql LOAD spatial;
%sql LOAD httpfs;
%sql SET s3_region = 'us-west-2';

- This part should output each category from overture categories.csv file
- SQL script to parse locations based on categories from a particular metropolitan region

In [ ]:
get_names_and_categories = """
SELECT
    id,
    names.primary AS primary_name,
    ST_AsText(geometry) AS geometry
FROM
    read_parquet('s3://overturemaps-us-west-2/release/2025-05-21.0theme=places/type=place/*', filename=true, hive_partitioning=1)
WHERE
    categories.primary = 'beer_garden'
"""